In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [4]:
shortforms = ['GSC', 'GSCs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [7]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [8]:
list(zip(longforms, counts))

[('glioma stem cells', 89),
 ('germline stem cell', 84),
 ('glioma stem cell', 72),
 ('glioblastoma stem cells', 44),
 ('glioblastoma stem cell', 38),
 ('glioma stem like cells', 29),
 ('glioma stem like cell', 27),
 ('glioblastoma stem like cell', 19),
 ('glioblastoma stem like cells', 11),
 ('gbm stem cells', 11),
 ('goosecoid', 10),
 ('gbm stem cell', 10),
 ('germ line stem cell', 9),
 ('gbm stem like cells', 7),
 ('gbm stem like cell', 5),
 ('germ line stem cells', 5),
 ('genomic standards consortium', 4),
 ('germ stem cells', 4),
 ('glut4 storage compartment', 3),
 ('glioma stemlike cell', 3),
 ('germ stem cell', 3),
 ('cm grown on germinated soybean', 3),
 ('glioma cscs', 3),
 ('green suspension cell cultures', 2),
 ('glioma stem initiating cell', 2),
 ('gc cancer stem cell', 2),
 ('goosecoid homeobox', 2),
 ('glioblastoma derived stem like cells', 2),
 ('gbm cancer stem cells', 2)]

In [9]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [10]:
names

{'MESH:D013234': 'Stem Cells',
 'GO:GO:0032593': 'insulin-responsive compartment',
 'GO:GO:0005623': 'cell',
 'MESH:D013025': 'Soybeans',
 'MESH:D018929': 'Cell Culture Techniques',
 'HGNC:4612': 'GSC'}

In [38]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [39]:
result = [grounding_map, names, pos_labels]

In [40]:
result

[{'cm grown on germinated soybean': 'BTO:BTO:0006345',
  'gbm cancer stem cells': 'BTO:BTO:0006345',
  'gbm stem cell': 'BTO:BTO:0006345',
  'gbm stem cells': 'BTO:BTO:0006345',
  'gbm stem like cell': 'BTO:BTO:0006345',
  'gbm stem like cells': 'BTO:BTO:0006345',
  'gc cancer stem cell': 'BTO:BTO:0006345',
  'genomic standards consortium': 'ungrounded',
  'germ line stem cell': 'MESH:D000072956',
  'germ line stem cells': 'MESH:D000072956',
  'germ stem cell': 'MESH:D000072956',
  'germ stem cells': 'MESH:D000072956',
  'germline stem cell': 'MESH:D000072956',
  'glioblastoma derived stem like cells': 'BTO:BTO:0006345',
  'glioblastoma stem cell': 'BTO:BTO:0006345',
  'glioblastoma stem cells': 'BTO:BTO:0006345',
  'glioblastoma stem like cell': 'BTO:BTO:0006345',
  'glioblastoma stem like cells': 'BTO:BTO:0006345',
  'glioma cscs': 'BTO:BTO:0006345',
  'glioma stem cell': 'BTO:BTO:0006345',
  'glioma stem cells': 'BTO:BTO:0006345',
  'glioma stem initiating cell': 'BTO:BTO:0006345',


In [60]:
grounding_map, names, pos_labels = [{'cm grown on germinated soybean': 'BTO:BTO:0006345',
  'gbm cancer stem cells': 'BTO:BTO:0006345',
  'gbm stem cell': 'BTO:BTO:0006345',
  'gbm stem cells': 'BTO:BTO:0006345',
  'gbm stem like cell': 'BTO:BTO:0006345',
  'gbm stem like cells': 'BTO:BTO:0006345',
  'gc cancer stem cell': 'BTO:BTO:0006345',
  'genomic standards consortium': 'ungrounded',
  'germ line stem cell': 'MESH:D000072956',
  'germ line stem cells': 'MESH:D000072956',
  'germ stem cell': 'MESH:D000072956',
  'germ stem cells': 'MESH:D000072956',
  'germline stem cell': 'MESH:D000072956',
  'glioblastoma derived stem like cells': 'BTO:BTO:0006345',
  'glioblastoma stem cell': 'BTO:BTO:0006345',
  'glioblastoma stem cells': 'BTO:BTO:0006345',
  'glioblastoma stem like cell': 'BTO:BTO:0006345',
  'glioblastoma stem like cells': 'BTO:BTO:0006345',
  'glioma cscs': 'BTO:BTO:0006345',
  'glioma stem cell': 'BTO:BTO:0006345',
  'glioma stem cells': 'BTO:BTO:0006345',
  'glioma stem initiating cell': 'BTO:BTO:0006345',
  'glioma stem like cell': 'BTO:BTO:0006345',
  'glioma stem like cells': 'BTO:BTO:0006345',
  'glioma stemlike cell': 'BTO:BTO:0006345',
  'glut4 storage compartment': 'GO:GO:0032593',
  'goosecoid': 'HGNC:4612',
  'goosecoid homeobox': 'HGNC:4612',
  'green suspension cell cultures': 'ungrounded'},
 {'BTO:BTO:0006345': 'glioblastoma multiforme cancer stem cell',
  'MESH:D000072956': 'Adult Germline Stem Cells',
  'GO:GO:0032593': 'insulin-responsive compartment',
  'HGNC:4612': 'GSC'},
 ['BTO:BTO:0006345', 'HGNC:4612', 'MESH:D000072956']]

In [61]:
excluded_longforms = []

In [62]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [63]:
additional_entities = {}

In [64]:
unambiguous_agent_texts = {"HGNC:4612": ["GSC", ["goosecoid"]]}

In [65]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [66]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [67]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [68]:
intersection1

[]

In [69]:
intersection2

[]

In [70]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [71]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [72]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-31 21:34:02] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-31 21:34:13] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.97186919472532 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [73]:
classifier.stats

{'label_distribution': {'MESH:D000072956': 99,
  'GO:GO:0032593': 3,
  'BTO:BTO:0006345': 447,
  'HGNC:4612': 25,
  'ungrounded': 3},
 'f1': {'mean': 0.971869, 'std': 0.004658},
 'precision': {'mean': 0.974512, 'std': 0.005242},
 'recall': {'mean': 0.975468, 'std': 0.003573},
 'MESH:D000072956': {'f1': {'mean': 0.979466, 'std': 0.018867},
  'pr': {'mean': 0.969474, 'std': 0.024943},
  'rc': {'mean': 0.99, 'std': 0.02}},
 'HGNC:4612': {'f1': {'mean': 0.714286, 'std': 0.071429},
  'pr': {'mean': 0.56, 'std': 0.08},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'BTO:BTO:0006345': {'f1': {'mean': 0.984579, 'std': 0.004122},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.969659, 'std': 0.007985}},
 'GO:GO:0032593': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}}}

In [74]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [75]:
disamb.dump(model_name, results_path)

In [76]:
print(disamb.info())

Disambiguation model for GSC, and GSCs

Produces the disambiguations:
	Adult Germline Stem Cells*	MESH:D000072956
	GSC*	HGNC:4612
	glioblastoma multiforme cancer stem cell*	BTO:BTO:0006345
	insulin-responsive compartment	GO:GO:0032593

Class level metrics:
--------------------
Grounding                               	Count	F1     
glioblastoma multiforme cancer stem cell*	447	0.98458
               Adult Germline Stem Cells*	 99	0.97947
                                     GSC*	 25	0.71429
          insulin-responsive compartment	  3	    0.6
                              Ungrounded	  3	    0.4

Weighted Metrics:
-----------------
	F1 score:	0.97187
	Precision:	0.97451
	Recall:		0.97547

* Positive labels
See Docstring for explanation



In [77]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1